# origin_race_dataテーブルに保存されているデータを最低限扱いやすいように変換する処理

In [1]:
import pandas as pd
import tqdm

import os
import sys
sys.path.append(os.pardir)
import common.com as common
from common import const,utils
import copy
from datetime import datetime


In [2]:
# origin_race_dataをpandasのDataFrameに保存
data = common.get_all_data(table_name='origin_race_data')
origin_data = pd.DataFrame(data, columns=const.ORIGIN_RACE_DATA_HEADERS)
converted_data = copy.deepcopy(origin_data)

In [3]:
rank_data_set = set(converted_data['着順'])
rank_data_set

# 着順に「降、失、中」など文字が含まれ、数値に変換不可である場合に無効な値（99）を設定
for data in tqdm.tqdm(rank_data_set):    
    if utils.isfloat(data) == False:
        converted_data['着順'] = converted_data['着順'].replace(data, const.INVALID_ORDER)
    else:
        converted_data['着順'] = converted_data['着順'].replace(data, int(data))

rank_data_set = set(converted_data['着順'])
rank_data_set

 93%|█████████▎| 27/29 [00:02<00:00, 19.14it/s]C:\Users\ionde\AppData\Local\Temp\ipykernel_32576\2278154381.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  converted_data['着順'] = converted_data['着順'].replace(data, int(data))
100%|██████████| 29/29 [00:02<00:00, 11.92it/s]


{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 99}

In [4]:
# 走破時間を秒数に変換
def convert_time(time):
    try:
        time_obj = datetime.strptime(time, "%M:%S.%f").time()
        time_seconds = time_obj.hour * 3600 + time_obj.minute * 60 + time_obj.second + time_obj.microsecond / 1000000
        return time_seconds
    except ValueError:
        return None
    

converted_data['走破時間'] = converted_data['走破時間'].map(lambda data: convert_time(str(data)))

In [5]:
distance_set=set(converted_data['距離'])

In [6]:
odds_set = set(converted_data['オッズ'])
for data in tqdm.tqdm(odds_set):    
    if utils.isfloat(data) == False:
        converted_data['オッズ'] = converted_data['オッズ'].replace(data, 0)
    else:
        converted_data['オッズ'] = converted_data['オッズ'].replace(data, float(data))

100%|█████████▉| 6814/6815 [10:42<00:00, 28.15it/s]C:\Users\ionde\AppData\Local\Temp\ipykernel_32576\3088802275.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  converted_data['オッズ'] = converted_data['オッズ'].replace(data, float(data))
100%|██████████| 6815/6815 [10:42<00:00, 10.60it/s]


In [7]:
converted_data['オッズ'] = converted_data['オッズ'].replace('---',0).astype(float)

In [8]:
converted_data['日付'] = converted_data['日付'].map(lambda data: datetime.strptime(data, "%Y年%m月%d日"))
converted_data['日付']

0        2023-01-05
1        2023-01-05
2        2023-01-05
3        2023-01-05
4        2023-01-05
            ...    
592324   2024-12-28
592325   2024-12-28
592326   2024-12-28
592327   2024-12-28
592328   2024-12-28
Name: 日付, Length: 592329, dtype: datetime64[ns]

In [ ]:
# データの保存
converted_data.to_csv("temp.csv", index=False)
common.insert_data("D:/work/Programming/scraping/temp.csv",table_name='race_data')
os.remove("temp.csv")